In [ ]:
#### FOR ACADEMIC PROJECT WORK
#check if lib install or not, if not, install

# !pip install wfdb
# !pip install lightgbm
# !pip install PyWavelets
# !pip install biosppy
# !pip install torch
# !pip install shap

In [ ]:
#unzip the data into the data folder
# import zipfile
# with zipfile.ZipFile('/workspaces/ecg-arrhythmia-model/data/train_dataset.zip', 'r') as zip_ref:
#     zip_ref.extractall('/workspaces/ecg-arrhythmia-model/data/')
# import os
# #check number of files exist in each data folders
# print('Number of files in each folder')
# print('cpsc_processed:', len(os.listdir('/workspaces/ecg-arrhythmia-model/data/cpsc_processed')))
# print('train_dataset:', len(os.listdir('/workspaces/ecg-arrhythmia-model/data/train_dataset')))
# print('test_dataset:', len(os.listdir('/workspaces/ecg-arrhythmia-model/data/test_dataset')))

In [1]:
import re
from datetime import datetime
import shutil
import os
from utils import load_dictionary_from_file
#load .hea file
file_name = 'data\ptb-xl\HR00001.hea'


In [ ]:
# DEBUG CODE - READ FILES

with open(file_name, 'r') as file:
    main_content = file.read()
# look for contains of the file between tag <code> and </code>
# and return the first match
# match = re.search(r'<code>(.*?)</code>', html_content, re.DOTALL)
if main_content:
    print(main_content)
else:
    print('No match')

In [ ]:
# DEBUG CODE - MODIFY FILES

#append date time to first line of match
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
# print(dt_string)
#split the match into lines
lines = main_content.split('\n')
#append date time to first line
lines[0] = lines[0] + ' ' + dt_string
#in the next 12 lines:

for i in range(1, 13):
    # lines[i] = lines[i][:12] + lines[i][14:] #remove 'x1' after 12th character
    # lines[i] = lines[i][:20] + lines[i][25:] #remove '.0(0)' after 20th character
    #get location of '.mat' in the line
    mat_loc = lines[i].find('.mat')+4 #location id at beginning of '.mat' -> +4 till the end
    # print(mat_loc)
    lines[i] = lines[i][:mat_loc+3] + lines[i][mat_loc+5:] #remove 'x1'
    lines[i] = lines[i][:mat_loc+11] + lines[i][mat_loc+16:] #remove '.0(0)'

#in the remaining lines, replace "#" with "# "
for i in range(13, len(lines)):
    lines[i] = lines[i].replace('# ', '#')


#join the lines back together
new_content = '\n'.join(lines)
print(new_content)

In [ ]:
#1. FUNCTION READ, AND MODIFY FILES
def process_file(file_name):
    with open(file_name, 'r') as file:
        main_content = file.read()
    # main_content = re.search(r'<code>(.*?)</code>', main_content, re.DOTALL) #no need to get part of the file
    file.close()
    if main_content:
        # lines = main_content.group(1).split('\n')
        lines = main_content.split('\n')
        now = datetime.now()
        dt_string = now.strftime("%d-%b-%Y %H:%M:%S")
        lines[0] = lines[0] + ' ' + dt_string #append date time to first line
        for i in range(1, 13):
            #get location of '.mat' in the line
            mat_loc = lines[i].find('.mat')+4 #location id at beginning of '.mat' -> +4 till the end
            # print(mat_loc)
            lines[i] = lines[i][:mat_loc+3] + lines[i][mat_loc+5:] #remove 'x1'
            lines[i] = lines[i][:mat_loc+11] + lines[i][mat_loc+16:] #remove '.0(0)'
        for i in range(13, len(lines)):
            lines[i] = lines[i].replace('# ', '#') #replace "#" with "# " in the remaining lines
        new_content = '\n'.join(lines)
        # new_file_name = file_name.replace('.hea', '_new.hea') #no need to change the file name
        #overwrite the original file
        with open(file_name, 'w') as file:
            file.write(new_content)
            file.close()
        # print('New file created:', file_name)
    else:
        print('No match')

In [ ]:
# CALL manual (read and modify files)
#loop through the files in main folder and process each file
files = os.listdir('data\CPSC_2018')
#make full path to each file
files = [os.path.join('data\CPSC_2018', file) for file in files]
for file in files:
    if file.endswith('.hea'):
        process_file(file)

In [ ]:
#2. FUNCTION MOVE FILES
def copy_files(folder, N):
    files = os.listdir(folder)
    #if N = 0, loop all files
    if N == 0:
        N = len(files)

    for i in range(N):
        file_name = os.path.join(folder, files[i])
        new_folder = os.path.dirname(folder) #copy to upper folder
        # new_folder = os.path.join(os.path.dirname(folder), 'main-data')
        shutil.copy(file_name, new_folder)
        # print('Copied:', file_name)
    print('Done, ', N, 'files copied in', folder)

# copy_files('.\data\CPSC_2018\g1', 50)

In [ ]:
#3. FUNCTION MOVE AND PROCESS FILES

def main_hea_process(data_folder, N):
    #make a list of sub-folders within the main data folder
    folders = os.listdir(data_folder)
    #loop through the folders and copy N files from each sub-folder
    for folder in folders:
        folder_name = os.path.join(data_folder, folder)
        if os.path.isdir(folder_name): #filter dirs only
            copy_files(folder_name, N)
    #loop through the files in main folder and process each file
    files = os.listdir(data_folder)
    #make full path to each file
    files = [os.path.join(data_folder, file) for file in files]
    for file in files:
        if file.endswith('.hea'):
            process_file(file)
    print('Done processing folders:', data_folder)


In [ ]:
#define main data folder string
data_folder = 'data\chapman_shaoxing'
# 'data\ptb-xl'
# 'data\cpsc_2018'
# 'data\cpsc_2018_extra'
# 'data\cpsc_processed'

#define samples get from each sub-folder
N = 0   #sampling files in each sub-folder.
        #Total file = N*number of sub-folder
        #N=0, get all files in each sub-folder

#process the main data folder
main_hea_process(data_folder, N)


In [ ]:
#Utility function to get unique codes from the files

def get_unique_dx_code (data_folder):
    files = os.listdir(data_folder)
    files = [os.path.join(data_folder, file) for file in files]
    list_codes = []
    for file in files:
        if file.endswith('.hea'):
            #read 16th line of the file
            with open(file, 'r') as f:
                lines = f.readlines()
                f.close()
            #get the 16th line
            line = lines[15]
            #get text after '#Dx: '
            text = line.split('#Dx: ')[1]
            #split the text into list of code, delimited by ','
            codes = text.split(',')
            #remove leading and trailing white spaces from each code
            codes = [code.strip() for code in codes]
            #append unique codes to the list
            for code in codes:
                if code not in list_codes:
                    list_codes.append(code)
    return list_codes

#call the function
# data_folder = 'data\cpsc_2018'
list_codes = get_unique_dx_code(data_folder)


In [ ]:
import json

# Function to load dictionary from file
def load_dictionary_from_file(filename):
    with open(filename, 'r') as file:
        dictionary = json.load(file)
    return dictionary

# Function to store dictionary to file
def store_dictionary_to_file(dictionary, filename):
    with open(filename, 'w') as file:
        json.dump(dictionary, file)

In [ ]:
# location of dx_dict file and class_labels file
file_dx_name = 'meta_data\dx_dict.json'  # File name to store the dictionary dx_dict
file_class_name = 'meta_data\class_labels.txt' # File name to store the class labels

# if dx_dict can be loaded from file, then load it, else use default dx_dict
if os.path.exists(file_dx_name):
    dx_dict = load_dictionary_from_file(file_dx_name)
else:
    #introduce dx_dict to store the codes
    dx_dict = {
            '426783006': 'SNR', # Normal sinus rhythm
            '164889003': 'AF', # Atrial fibrillation
            '270492004': 'IAVB', # First-degree atrioventricular block
            '164909002': 'LBBB', # Left bundle branch block
            '713427006': 'RBBB', # Complete right bundle branch block
            '59118001': 'RBBB', # Right bundle branch block
            '284470004': 'PAC', # Premature atrial contraction
            '63593006': 'PAC', # Supraventricular premature beats
            '164884008': 'PVC', # Ventricular ectopics
            '429622005': 'STD', # ST-segment depression
            '164931005': 'STE', # ST-segment elevation
        }


In [ ]:
#append the codes to the dictionary
for code in list_codes:
    if code not in dx_dict:
        # print('New code found:', code)
        #introduce new id sequence for new codes
        seq = 'Other' + str(len(dx_dict)-11+1) #original codes are 11
        dx_dict[code] = seq
# print(dx_dict)


In [ ]:
#check duplicate codes in dx_dict

#make a list of codes
codes = list(dx_dict.keys())
#make a list of unique codes
unique_codes = list(set(codes))
#check if the length of the two lists are the same
if len(codes) == len(unique_codes):
    print('No duplicate codes')
else:
    print('Duplicate codes found')
    #loop through the unique codes
    for code in unique_codes:
        #count the number of times the code appears in the list of codes
        count = codes.count(code)
        #if the count is greater than 1, print the code and the count
        if count > 1:
            print(code, count)

In [ ]:
# store new dx_dict to file
dx_dict = {
    '426783006': 'SNR', # Normal sinus rhythm
    '164889003': 'AF', # Atrial fibrillation
    '270492004': 'IAVB', # First-degree atrioventricular block
    '164909002': 'LBBB', # Left bundle branch block
    '713427006': 'RBBB', # Complete right bundle branch block
    '59118001': 'RBBB', # Right bundle branch block
    '284470004': 'PAC', # Premature atrial contraction
    '63593006': 'PAC', # Supraventricular premature beats
    '164884008': 'PVC', # Ventricular ectopics
    '429622005': 'STD', # ST-segment depression
    '164931005': 'STE', # ST-segment elevation
}
# filename = os.path.join(data_folder, filename)
store_dictionary_to_file(dx_dict, file_dx_name)

In [ ]:
#define function to make class labels from the dictionary

#function to get unique values from the dictionary -> as class labels
def get_unique_values(dictionary):
    values = list(dictionary.values())
    unique_values = list(set(values))
    return unique_values

#function to store unique values to file
def store_class_label_to_file(values, filename):
    with open(filename, 'w') as file:
        for value in values:
            file.write(value + '\n')
        file.close()

#function to load unique values from file and store in a list
def load_class_label_from_file(filename):
    class_label = []
    with open(filename, 'r') as file:
        for line in file:
            class_label.append(line.strip())
    return class_label

In [ ]:
file_class_name = 'meta_data\class_labels.txt' # File name to store the class labels
#get class labels from dictionary

class_labels = get_unique_values(dx_dict)
#store class labels to file
store_class_label_to_file(class_labels, file_class_name)

#load dict back to file
#dictionary = load_dictionary_from_file(filename)
#load class labels back to file
#class_labels = load_class_label_from_file(filename)

In [ ]:
#load dict back to file
dictionary = load_dictionary_from_file(filename)

In [ ]:
#Script to execute the above functions

# 'data\ptb-xl'
# 'data\cpsc_2018'
# 'data\cpsc_2018_extra'
# 'data\cpsc_processed'


#python preprocess.py --data-dir 'data\cpsc_2018_extra'
#python baselines.py --data-dir 'data\cpsc_2018_extra'  --classifier 'LR'
#python main.py --data-dir 'data\ptb-xl' --leads 'all' --epochs 2 --use-gpu --batch-size 200
#python predict.py --data-dir "data\cpsc_processed" --leads 'all' --use-gpu --batch-size 64

In [ ]:
#def function to move files .hea and .mat from one folder to another.
#Files name are extracted from 1st column of df
def move_files(df, source_folder, dest_folder):
    for index, row in df.iterrows():
        file_name = row['File']
        #append '.hea' and '.mat' to the file name, and move the files, one by one
        for ext in ['.hea', '.mat']:
            source_file = os.path.join(source_folder, file_name + ext)
            dest_file = os.path.join(dest_folder, file_name + ext)
            shutil.move(source_file, dest_file)
    print('Done moving files')

In [ ]:
# %run "preprocess.py" --data-dir "data/cpsc_processed" -- done in full
# %run "preprocess.py" --data-dir "data/train_dataset" -- done in full

#run function move file and handle .hea files
# %run "preprocess.py" --data-dir "data\chapman_shaoxing"
# %run "preprocess.py" --data-dir "data\test_dataset"
# %run "baselines.py" --data-dir "data\train_dataset" --classifier 'all'
# %run "main.py" --data-dir "data\cpsc_2018_extra" --leads 'all' --epochs 1 --batch-size 200

In [ ]:
%run "baselines.py" --data-dir "data/cpsc_processed" --classifier 'all'

In [ ]:
%run "baselines.py" --data-dir "data/train_dataset" --classifier 'all'

In [ ]:
#resnet34, testset is part of trainset
%run "main.py" --data-dir "data/cpsc_processed" --epochs 2 --num-workers 2 --batch-size 8 --phase 'test'
#--epochs 4 --use-gpu --num-workers 2
#--phase 'train' ('test')

In [ ]:
#resnet34 test, testset is part of train_dataset 
%run "main.py" --data-dir "data/train_dataset" --epochs 2 --num-workers 2 --batch-size 8 --phase 'test'


In [2]:
#resnet34, testset is new test datase, using labels_altered.csv
%run "main.py" --data-dir "data/cpsc_processed" --epochs 2 --num-workers 2 --batch-size 8 --phase 'test'

Validating...


100%|██████████| 348/348 [06:45<00:00,  1.16s/it]


Loss: 150.4477
Shape of y_trues: (2782, 9)
Shape of y_scores: (2782, 9)
F1s: [0.81541409 0.93687708 0.63247863 0.76571429 0.70489039 0.67953668
 0.74545455 0.42080378 0.75      ]
Avg F1: 0.7168
AUCs: [0.86476461 0.99243247 0.93949008 0.97670868 0.97025251 0.85062641
 0.95459687 0.80194205 0.95711364]
Avg AUC: 0.9231


In [2]:
#resnet34 train, epochs 5, batch size 8, num workers 2
%run "main.py" --data-dir "data/train_dataset" --epochs 3 --num-workers 2 --batch-size 8

Training epoch 0:


100%|██████████| 2783/2783 [2:54:01<00:00,  3.75s/it]  


Loss: 492.1525
Validating...


100%|██████████| 696/696 [13:14<00:00,  1.14s/it]


Loss: 90.0963
Shape of y_trues: (5566, 9)
Shape of y_scores: (5566, 9)
F1s: [0.92325764 0.8582996  0.5963939  0.84142395 0.8256513  0.41702128
 0.77697842 0.37609329 0.25806452]
Avg F1: 0.6526
Training epoch 1:


...

Loss: 75.1440
Shape of y_trues: (5566, 9)
Shape of y_scores: (5566, 9)
F1s: [0.93762158 0.91855584 0.67651403 0.87868852 0.85912698 0.52014652
 0.80046948 0.46054054 0.27826087]
Avg F1: 0.7033
Training epoch 2:


100%|██████████| 2783/2783 [2:54:20<00:00,  3.76s/it]  


Loss: 291.0483
Validating...


100%|██████████| 696/696 [13:15<00:00,  1.14s/it]


Loss: 67.4626
Shape of y_trues: (5566, 9)
Shape of y_scores: (5566, 9)
F1s: [0.94713426 0.9279732  0.65426357 0.84542587 0.86359176 0.51503759
 0.82978723 0.51840491 0.44705882]
Avg F1: 0.7276
